In [36]:
import math
import numpy as np
import xml.etree.ElementTree as ET
import scipy.optimize as scpo

# Задание 1 - (испарение капли)
## Выполнил Васин Антон
## Вариант 12

### 1)Считывание файла

In [37]:
input_file = open("input.txt", 'r', encoding="utf-8")
data = input_file.readlines()
input_file.close()

### Присваивание параметров

In [38]:
R = 8.314
N_a = 6.022e23
k_b = 1.380649e-23;
    
#NH3
#При нормальных условиях
ro_fluid = float(data[2])
T_boil = float(data[4])
print(T_boil)
H_vap = float(data[6])
M = float(data[8])

#При P = 2атм условиях
T_boil = 1/((1/T_boil)-((R*math.log(2))/(H_vap*M)))
print(T_boil)
    
# Атмосфера
lambda_g1 = float(data[12])
lambda_g2 = float(data[14])
T = float(data[16])
P = float(data[18])
M_a = float(data[20])
C_p = float(data[22])
X_inf = float(data[24])

# Diameter
D = float(data[27])
Dif = float(data[29])

239.81
254.90860230810287


In [39]:
print("ro_fluid =",ro_fluid)
print("T_boil =",round(T_boil, 1))
print("H_vap =",H_vap)
print("M =",M)
print("lambda_g1 =",lambda_g1)
print("lambda_g2 =",lambda_g2)
print("T =",T)
print("P =",P)
print("M_a =",M_a)
print("C_p =",C_p)
print("X_inf =",X_inf)
print("D =",D)
print("Dif =",Dif)

ro_fluid = 681.4
T_boil = 254.9
H_vap = 1370000.0
M = 0.0170306
lambda_g1 = 0.0421
lambda_g2 = 0.056
T = 523.15
P = 202650.0
M_a = 0.02898
C_p = 1034.0
X_inf = 0.1
D = 0.008
Dif = 3e-05


### 2)Вычисление Основных велечин в начальный момент

In [40]:
V = math.pi * D ** 3 / 2.0 / 3.0
S = math.pi * D ** 2
m = ro_fluid * V
mols = m / M
P_vapor_inf = P * X_inf
Y_inf = (X_inf*M)/(X_inf*M + (1.0 - X_inf)*M_a)
print("Y_inf =", round(Y_inf, 5))

Y_inf = 0.06129


### 3)Вычисление параметров на поверхности

In [41]:
def T_w_equation(T_w):
    X_w = math.exp((H_vap * M / R) * (1.0 / T_boil - 1.0 / T_w))
    Y_w = (X_w*M)/(X_w*M + (1.0 - X_w)*M_a)
    return T - (H_vap/C_p)*((Y_w - Y_inf)/(1.0 - Y_w)) - T_w

T_w= scpo.fsolve(T_w_equation, T_boil - 1)[0]
X_w = math.exp((H_vap * M / R) * (1.0 / T_boil - 1.0 / T_w))
Y_w = (X_w*M)/(X_w*M + (1.0 - X_w)*M_a)
P_w = P * X_w

mu = M*X_w + M_a*(1 - X_w)
ro_w = (P * mu) / (R * T_w)
lambda_g = (lambda_g2**X_w)*(lambda_g1**(1-X_w))
dm_dt_S = lambda_g / (C_p * (D/2.0)) * math.log(1.0 + C_p * (T - T_w)/ H_vap )
v_w = dm_dt_S/ro_w
dm_dt = dm_dt_S * S

print('T_w =', round(T_w, 5), "K")
print('X_w =', round(X_w, 5))
print('Y_w =', round(Y_w, 5))
print('P_w =', round(P_w, 5))
print('dm_dt_S =', dm_dt_S)
print('ro_w =', ro_w)
print('v_w =', v_w)
print('dm_dt =', dm_dt)

T_w = 232.0172 K
X_w = 0.3375
Y_w = 0.2304
P_w = 68393.94807
dm_dt_S = 0.002226203927303512
ro_w = 2.6208159275730236
v_w = 0.0008494316231377083
dm_dt = 4.476048578182054e-07


### Время испарения капли

In [42]:
t_0 = ro_fluid * C_p * (D / 2.0) ** 2 / 2 / lambda_g / math.log(1 + C_p / H_vap * (T - T_w))
print('t_0 =', t_0)

t_0 = 612.1631461007664


### 4)Запись параметров поверхности в файл

In [43]:
    L = [f"Температура поверхности 𝑇𝑤 [K]\n\t{ round(T_w, 4)}\n",
         f"Мольная доля пара на поверхности 𝑋𝑤\n\t{round(X_w, 6)}\n",
         f"Массовая доля пара на поверхности 𝑌𝑤\n\t{round(Y_w, 6)}\n",
         f"Плотность газа на поверхности 𝜌𝑤 [кг/м3]\n\t{round(ro_w, 2)}\n",
         f"Скорость пара на поверхности 𝑣𝑤 [мм/с]\n\t{round(v_w, 12) * 1000}\n",
         f"Удельный массовый поток с единицы поверхности [г/см^2·с]\n\t{round(dm_dt_S, 14) / 10}\n",
         f"Массовая скорость испарения в начальный момент времени [г/с]\n\t{round(dm_dt, 17) * 1000}\n",
         f"Время полного испарения капли 𝑡0 [c]\n\t{round(t_0, 2)}\n",
        f"Теплопроводность смеси у поверхности [вт/м*K]]\n\t{round(lambda_g, 5)}\n"]
    output_file = open("output_1.txt", 'w', encoding="utf-8")
    output_file.writelines(L)
    output_file.close()

### 5)Вычисление радиальных зависимостей

In [44]:
N = 1900
R_max = D / 2.0 * 20.0
r = np.linspace(D / 2.0, R_max, N+1)
B = C_p * dm_dt_S * (D / 2.0) ** 2 / lambda_g
C_1 = (T - T_w) / (1 - math.exp(- 2 * B / D))
T_r = C_1 * np.exp(- B / r) + T - C_1

##Старый метод
Y_r = (Y_inf + C_p / H_vap * (T - T_r)) / (1.0 + C_p / H_vap * (T - T_r))
print(Y_r[500])

##Новый метод
##1.34 - плотность воздуха
##Dif - коэффициент дифузии в чистом воздухе
B_1 = (dm_dt_S * (D / 2.0) ** 2) / (1.34*Dif)
C_1_1 = (Y_inf - Y_w) / (1 - math.exp(- 2 * B_1 / D))
Y_r = C_1_1 * np.exp(- B_1 / r) + Y_inf - C_1_1
print(Y_r[500])

a = (ro_w * v_w * D**2)/ 4.0
X_r = Y_r / (M * (Y_r / M + (1.0 - Y_r) / M_a))
mu = (X_r * M + (1.0 - X_r) * M_a)
ro_r = P * mu / (T_r * R)

print(ro_r[0])
print(ro_r[-1])
print("Плотность сильно меняется от радиуса поэтому считать зависимость Мольной доли от радиуса анологичной Температуре неверно")
v_g_r = (a) / (ro_r * (r**2))
r = r*1000
v_g_r = v_g_r*10**3

0.09715350912971343
0.09214206068762376
2.620815927573026
1.3269552861963023
Плотность сильно меняется от радиуса поэтому считать зависимость Мольной доли от радиуса анологичной Температуре неверно


### 6)Запись в файл радиальных зависимостей

In [45]:
L_r = ["r[mm], Y_v[-], T_g[K], V_g[mkm/s]\n"] + [f"{r[i]}, {Y_r[i]}, {T_r[i]}, {v_g_r[i]}\n" for i in range(N+1)]
output_file_r = open("output_r.txt", 'w', encoding="utf-8")
output_file_r.writelines(L_r)
output_file_r.close()